Ok - new tree with all isolates

In [1]:
import glob
import re

Ok - trying to use prokka - roary to make a pangenome

Made a conda environment conda create --name cpowgs
Then conda install -c conda-forge -c bioconda -c defaults prokka
then
conda config --add channels r\
conda config --add channels defaults\
conda config --add channels conda-forge\
conda config --add channels bioconda\
conda install roary\
then\
conda install jupyterlab (I know, wrong order)\
then\
conda install raxml\
then\
conda install snp-sites\
then\
conda install pandas\
then\
conda install rpy2\
then\
conda install r-tidyverse\

In [2]:
%cd /atium/Data/Nanopore/cpowgs/trees/200811_timp_tree

/atium/Data/Nanopore/cpowgs/trees/200811_timp_tree


Ok - I need to pull down the genbank files for Enterobacter cloacae: https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/025/565/GCF_000025565.1_ASM2556v1/GCF_000025565.1_ASM2556v1_genomic.gbff.gz
E. Coli: https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/005/845/GCF_000005845.2_ASM584v2/GCF_000005845.2_ASM584v2_genomic.gbff.gz
and K. Pneumo: https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/240/185/GCF_000240185.1_ASM24018v2/GCF_000240185.1_ASM24018v2_genomic.gbff.gz

In [3]:
%mkdir refs
%cd refs
!wget https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/005/845/GCF_000005845.2_ASM584v2/GCF_000005845.2_ASM584v2_genomic.gbff.gz
!gunzip *gz
%mv *gbff ecoli_ref.gb
!wget https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/025/565/GCF_000025565.1_ASM2556v1/GCF_000025565.1_ASM2556v1_genomic.gbff.gz
!gunzip *gz
%mv *gbff encl_ref.gb
!wget https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/240/185/GCF_000240185.1_ASM24018v2/GCF_000240185.1_ASM24018v2_genomic.gbff.gz
!gunzip *gz
%mv *gbff kpn_ref.gb
%cd ..

/atium/Data/Nanopore/cpowgs/trees/200811_timp_tree/refs
--2020-08-11 08:47:29--  https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/005/845/GCF_000005845.2_ASM584v2/GCF_000005845.2_ASM584v2_genomic.gbff.gz
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.13, 2607:f220:41e:250::7, 2607:f220:41e:250::13, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.13|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3462090 (3.3M) [application/x-gzip]
Saving to: ‘GCF_000005845.2_ASM584v2_genomic.gbff.gz’

GCF_000005845.2_ASM 100%[===================>]   3.30M  9.27MB/s    in 0.4s    

2020-08-11 08:47:29 (9.27 MB/s) - ‘GCF_000005845.2_ASM584v2_genomic.gbff.gz’ saved [3462090/3462090]

--2020-08-11 08:47:30--  https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/025/565/GCF_000025565.1_ASM2556v1/GCF_000025565.1_ASM2556v1_genomic.gbff.gz
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.13, 2607:f220:41e:250::7, 2607:f220:4